In [1]:
import pandas as pd
import numpy as np

# desabilitar avisos
import warnings
warnings.filterwarnings('ignore')

# 1. Import data

In [2]:
file_name = 'SEDIMENTO DE CORRENTE' # será usado para salvar o arquivo processado
df = pd.read_excel('Data/' + file_name + '.xlsx')

In [3]:
# Show all collumns
print(df.columns.to_list())

['PROJETO_AMOSTRAGEM', 'PROJETO_PUBLICACAO', 'CENTRO_DE_CUSTO', 'CLASSE', 'NUMERO_DE_CAMPO', 'NUMERO_DE_LABORATORIO', 'DUPLICATA', 'LATITUDE', 'LONGITUDE', 'LOTE', 'RA', 'DATA_DE_ANALISE', 'METODO', 'ABERTURA', 'LEITURA', 'LABORATORIO', 'JOB', 'Ag_ppb', 'Ag_ppm', 'Al_pct', 'Al2O3_pct', 'As_ppm', 'Au_ppb', 'Au1_ppb', 'Au2_ppb', 'Au_ppm', 'B_ppm', 'Ba_ppm', 'BaO_pct', 'Be_ppm', 'Bi_ppm', 'C_organico_pct', 'C_elementar_pct', 'Ca_pct', 'CaO_pct', 'Cd_ppm', 'Ce_ppm', 'Cl_ppm', 'Co_ppm', 'CO3_pct', 'Cr_pct', 'Cr_ppm', 'Cr2O3_pct', 'Cs_ppm', 'Cu_ppm', 'Dy_ppm', 'Er_ppm', 'Eu_ppm', 'F_ppm', 'Fe_pct', 'Fe_ppm', 'Fe2O3_pct', 'FeO_pct', 'Ga_ppm', 'Gd_ppm', 'Ge_ppm', 'Hf_ppm', 'Hg_ppb', 'Hg_ppm', 'Ho_ppm', 'In_ppm', 'K_pct', 'K2O_pct', 'La_ppm', 'Li_ppm', 'Lu_ppm', 'Mg_pct', 'MgO_pct', 'Mn_pct', 'Mn_ppm', 'MnO_pct', 'Mo_ppm', 'Na_pct', 'Na2O_pct', 'Nb_ppm', 'Nb2O5_pct', 'Nd_ppm', 'Ni_pct', 'Ni_ppm', 'P_pct', 'P_ppm', 'P2O5_pct', 'Pb_ppb', 'Pb_ppm', 'Pd_ppm', 'Pd_ppb', 'PF_pct', 'Pr_ppm', 'Pt_ppb',

# 2. Organizing data

## 2.1. ID columns

As colunas `ID_AMOSTRA` e `ID_REGISTRO` são colocadas após a coluna `JOB`. Encontre o índice da coluna `JOB` e adicione 1 e 2, respectivamente. <br>
<font color='gray'>The `ID_AMOSTRA` and `ID_REGISTRO` columns are placed after the `JOB` column. Find the index of the `JOB` column and add 1 and 2, respectively.</font>

In [4]:
# Get reference position
base_pos = df.columns.get_loc('JOB')

# Insert ID_AMOSTRA at JOB + 1
df.insert(base_pos + 1, 'ID_AMOSTRA', df['NUMERO_DE_CAMPO'])

# Insert ID_REGISTRO at JOB + 2
df.insert(base_pos + 1, 'ID_REGISTRO', df['NUMERO_DE_CAMPO'] + '_' + df['METODO'] + '_' + df['JOB'])

## 2.2. Remove duplicates

### Filtering by DUPLICATA column

In [5]:
df['DUPLICATA'].unique() #Verificar como são classificadas as duplicatas

array([nan, 'Sim'], dtype=object)

In [6]:
# Filtra somente os dados que a 'DUPLICATA' não é 'Sim'
# Filtering the dataframe to keep only rows where 'DUPLICATA' is not 'Sim'
df_duplicata = df[(df['DUPLICATA'] == 'Sim')]
df = df[~(df['DUPLICATA'] == 'Sim')]

### Filterin by duplicate ID_AMOSTRA

In [7]:
# Identify rows with duplicate ID_AMOSTRA
duplicates_count = df['ID_AMOSTRA'].duplicated().sum()

if duplicates_count > 0:
    print(f'Warning: {duplicates_count} duplicate IDs found!')
    # Show the duplicated rows for inspection
    df_duplicates = df[df['ID_AMOSTRA'].duplicated(keep=False)]
    print(df_duplicates.sort_values(by='ID_AMOSTRA'))
else:
    print('Success: No duplicate ID_AMOSTRA found.')

Success: No duplicate ID_AMOSTRA found.


# 3. Limit of detection (LOD), NA and zero values

## 3.1. Filtering data

Identifica e quantifica a presença de valores relacionados a Limites de Detecção (LOD), zeros e nulos. Os resultados são divididos em duas saídas: uma lista de colunas totalmente vazias e uma tabela mostrando a porcentagem de valores 'não ideais' por elemento <br>
<font color='gray'>Identify and quantify the presence of values related to Limits of Detection (LOD), zeros, and nulls. The results are split into two outputs: a list of entirely empty columns and a table showing the percentage of 'non-ideal' values per element.</font>


In [8]:
# Identificar colunas numéricas (elementos) e colunas totalmente vazias
# Identify empty columns
cols_elementos = [col for col in df.columns if '_' in col]
colunas_vazias = [col for col in cols_elementos if df[col].isnull().all()]
colunas_com_dados = [col for col in cols_elementos if col not in colunas_vazias]

# Saída 1: Lista de colunas vazias
print('--- EMPTY COLUMNS | COLUNAS VAZIAS ---')
if colunas_vazias:
    print(colunas_vazias)
else:
    print('No entirely empty columns found.')

print('\n')

--- EMPTY COLUMNS | COLUNAS VAZIAS ---
['Ag_ppb', 'Al2O3_pct', 'Au_ppb', 'Au1_ppb', 'Au2_ppb', 'BaO_pct', 'C_organico_pct', 'C_elementar_pct', 'CaO_pct', 'Cl_ppm', 'CO3_pct', 'Cr_pct', 'Cr2O3_pct', 'Dy_ppm', 'Er_ppm', 'Eu_ppm', 'F_ppm', 'Fe_ppm', 'Fe2O3_pct', 'FeO_pct', 'Gd_ppm', 'Hg_ppb', 'Ho_ppm', 'K2O_pct', 'MgO_pct', 'Mn_pct', 'MnO_pct', 'Na2O_pct', 'Nb2O5_pct', 'Nd_ppm', 'Ni_pct', 'P_pct', 'P2O5_pct', 'Pb_ppb', 'Pd_ppm', 'Pd_ppb', 'PF_pct', 'Pr_ppm', 'Pt_ppb', 'Pt_ppm', 'Re_ppb', 'Rh_ppb', 'SiO2_pct', 'Sm_ppm', 'Soma_pct', 'Ti_ppm', 'TiO2_pct', 'Tm_ppm']




In [9]:
# Remover colunas vazias
# Remove empty columns

df.drop(columns=colunas_vazias, inplace=True)

print(f'Removed {len(colunas_vazias)} empty columns.')

Removed 48 empty columns.


In [10]:
# Função para identificar o que é 'LOD' ou dado problemático
# Identify LOD value
def contar_lod(series):
    # Converte para string para buscar símbolos, mas mantém análise de nulos/zeros
    total = len(series)
    # Contagem de nulos, zeros e strings com < ou >
    mascara_lod = (
        series.astype(str).str.contains('<|>') | 
        (series == 0) | 
        series.isna()
    )
    soma = mascara_lod.sum()
    return (soma / total) * 100 if soma > 0 else 0

# Gerar resultados
resultados = {}
for col in colunas_com_dados:
    porcentagem = contar_lod(df[col])
    if porcentagem > 0:
        resultados[col] = f'{porcentagem:.2f}%'

print('--- LOD & NULL SUMMARY (%) | RESUMO DE LOD E NULOS (%) ---')
if resultados:
    df_resumo = pd.DataFrame.from_dict(resultados, orient='index', columns=['LOD/Null %'])
    print(df_resumo)
else:
    print('No LOD or null values detected in data-containing columns.')

--- LOD & NULL SUMMARY (%) | RESUMO DE LOD E NULOS (%) ---
       LOD/Null %
Ag_ppm    100.00%
Al_pct      0.56%
As_ppm      8.10%
Au_ppm    100.00%
B_ppm     100.00%
Bi_ppm      0.28%
Ca_pct     21.79%
Cd_ppm     18.16%
Fe_pct      0.56%
Ge_ppm     99.44%
Hf_ppm     31.56%
Hg_ppm    100.00%
In_ppm     14.25%
K_pct       0.84%
Li_ppm      1.12%
Lu_ppm     27.65%
Mg_pct     16.48%
Mo_ppm      2.23%
Na_pct     91.06%
Nb_ppm      9.22%
Ni_ppm    100.00%
P_ppm     100.00%
Rb_ppm     70.67%
Re_ppm    100.00%
S_pct      26.54%
Sb_ppm     68.16%
Sc_ppm      5.31%
Se_ppm     27.65%
Sn_ppm     71.79%
Ta_ppm      5.03%
Tb_ppm    100.00%
Th_ppm     57.26%
Ti_pct    100.00%
Tl_ppm    100.00%
V_ppm      65.08%
W_ppm      25.42%
Yb_ppm     94.97%
Zr_ppm      0.28%


In [11]:
# 1. Identificar colunas com mais de 30% de valores problemáticos
# Identify columns with more than 30% of LOD and NA values
colunas_para_remover = []

for col in colunas_com_dados:
    porcentagem = contar_lod(df[col])
    if porcentagem > 30:
        # Armazena o nome da coluna e a porcentagem formatada
        colunas_para_remover.append(f'{col}: {porcentagem:.2f}%')

# 2. Remover as colunas do DataFrame
# Extraímos apenas o nome (antes do ':') para poder dar o drop
nomes_para_drop = [item.split(':')[0] for item in colunas_para_remover]
df.drop(columns=nomes_para_drop, inplace=True)

print(f'--- REMOVAL SUMMARY | RESUMO DE REMOÇÃO ---')
if colunas_para_remover:
    print(f'Removed {len(colunas_para_remover)} columns with >30% LOD/Nulls:')
    for item in colunas_para_remover:
        print(item)
else:
    print('No columns exceeded the 30% threshold.')

--- REMOVAL SUMMARY | RESUMO DE REMOÇÃO ---
Removed 19 columns with >30% LOD/Nulls:
Ag_ppm: 100.00%
Au_ppm: 100.00%
B_ppm: 100.00%
Ge_ppm: 99.44%
Hf_ppm: 31.56%
Hg_ppm: 100.00%
Na_pct: 91.06%
Ni_ppm: 100.00%
P_ppm: 100.00%
Rb_ppm: 70.67%
Re_ppm: 100.00%
Sb_ppm: 68.16%
Sn_ppm: 71.79%
Tb_ppm: 100.00%
Th_ppm: 57.26%
Ti_pct: 100.00%
Tl_ppm: 100.00%
V_ppm: 65.08%
Yb_ppm: 94.97%


## 3.2. Replace LOD values

In [12]:
import re

# Dicionários para armazenar os limites detectados
# Dictionaries to store detected limits
lod_min_dict = {}
lod_max_dict = {}

def tratar_geochem_lod(valor, col_name):
    if pd.isna(valor) or not isinstance(valor, str):
        return valor
    
    valor_ajustado = valor.replace(',', '.')
    
    # 2. Trata Limite de Detecção Inferior (ex: <0,5) -> LD/2
    # 2. Handle Lower Detection Limit (e.g., <0.5) -> LOD/2
    if '<' in valor_ajustado:
        try:
            num = float(re.sub(r'[^\d.]', '', valor_ajustado))
            # Armazena o valor do limite original (antes de dividir por 2)
            # Stores the original limit value (before dividing by 2)
            lod_min_dict[col_name] = num
            return num / 2
        except ValueError:
            return valor
            
    # 3. Trata Limite de Detecção Máximo (ex: >500) -> Valor do Limite
    # 3. Handle Maximum Detection Limit (e.g., >500) -> Limit Value
    elif '>' in valor_ajustado:
        try:
            num = float(re.sub(r'[^\d.]', '', valor_ajustado))
            # Armazena o valor do limite máximo
            # Stores the maximum limit value
            lod_max_dict[col_name] = num
            return num
        except ValueError:
            return valor
            
    try:
        return float(valor_ajustado)
    except ValueError:
        return valor

# Aplicando a função coluna por coluna para capturar o contexto (nome da coluna)
# Applying the function column by column to capture context (column name)
cols_analise = [col for col in df.columns if '_' in col]

for col in cols_analise:
    # Passamos o nome da coluna para a função via lambda
    df[col] = df[col].apply(lambda x: tratar_geochem_lod(x, col))

print('LOD substitution completed | Substituição de LOD concluída.')
print(f'Detected Min LODs: {lod_min_dict}')
print(f'Detected Max LODs: {lod_max_dict}')

LOD substitution completed | Substituição de LOD concluída.
Detected Min LODs: {'Al_pct': 0.01, 'As_ppm': 1.0, 'Bi_ppm': 0.02, 'Ca_pct': 0.01, 'Cd_ppm': 0.01, 'Fe_pct': 0.01, 'In_ppm': 0.02, 'K_pct': 0.01, 'Li_ppm': 1.0, 'Lu_ppm': 0.01, 'Mg_pct': 0.01, 'Mo_ppm': 0.05, 'Nb_ppm': 0.05, 'S_pct': 0.01, 'Sc_ppm': 0.1, 'Se_ppm': 1.0, 'Ta_ppm': 0.05, 'W_ppm': 0.1, 'Zr_ppm': 0.5}
Detected Max LODs: {}


In [13]:
# testes = ['10,5', '0,25', '1000,0', '5,2']
# for t in testes:
#     resultado = tratar_geochem_lod(t)
#     print(f'Original: {t} -> Convertido: {resultado} (Tipo: {type(resultado)})')

## 3.3. Negative mumbers

In [14]:
# Show all collumns
print(df.columns.to_list())

['PROJETO_AMOSTRAGEM', 'PROJETO_PUBLICACAO', 'CENTRO_DE_CUSTO', 'CLASSE', 'NUMERO_DE_CAMPO', 'NUMERO_DE_LABORATORIO', 'DUPLICATA', 'LATITUDE', 'LONGITUDE', 'LOTE', 'RA', 'DATA_DE_ANALISE', 'METODO', 'ABERTURA', 'LEITURA', 'LABORATORIO', 'JOB', 'ID_REGISTRO', 'ID_AMOSTRA', 'Al_pct', 'As_ppm', 'Ba_ppm', 'Be_ppm', 'Bi_ppm', 'Ca_pct', 'Cd_ppm', 'Ce_ppm', 'Co_ppm', 'Cr_ppm', 'Cs_ppm', 'Cu_ppm', 'Fe_pct', 'Ga_ppm', 'In_ppm', 'K_pct', 'La_ppm', 'Li_ppm', 'Lu_ppm', 'Mg_pct', 'Mn_ppm', 'Mo_ppm', 'Nb_ppm', 'Pb_ppm', 'S_pct', 'Sc_ppm', 'Se_ppm', 'Sr_ppm', 'Ta_ppm', 'Te_ppm', 'U_ppm', 'W_ppm', 'Y_ppm', 'Zn_ppm', 'Zr_ppm', 'OBSERVACAO']


In [15]:
# 1. Seleciona apenas colunas dos elementos
# 1. Select only element columns
element_list = ['Al_pct', 'As_ppm', 'Ba_ppm', 'Be_ppm', 'Bi_ppm', 'Ca_pct', 'Cd_ppm', 'Ce_ppm', 'Co_ppm', 'Cr_ppm', 'Cs_ppm', 'Cu_ppm',
                'Fe_pct', 'Ga_ppm', 'In_ppm', 'K_pct', 'La_ppm', 'Li_ppm', 'Lu_ppm', 'Mg_pct', 'Mn_ppm', 'Mo_ppm', 'Nb_ppm', 'Pb_ppm', 'S_pct',
                'Sc_ppm', 'Se_ppm', 'Sr_ppm', 'Ta_ppm', 'Te_ppm', 'U_ppm', 'W_ppm', 'Y_ppm', 'Zn_ppm', 'Zr_ppm']

# 2. Identificar as linhas com valores negativos antes da remoção
# 2. Identify rows with negative values before removal
rows_to_drop = df[(df[element_list] < 0).any(axis=1)].index

# 3. Remover as linhas do DataFrame
# 3. Drop the rows from the DataFrame
if len(rows_to_drop) > 0:
    df.drop(index=rows_to_drop, inplace=True)
    print(f'--- REMOVAL SUMMARY | RESUMO DE REMOÇÃO ---')
    print(f'Removed {len(rows_to_drop)} rows containing negative values.')
    # Removed {len(rows_to_drop)} linhas contendo valores negativos.
else:
    print('No negative values found. No rows were removed.')
    # Nenhum valor negativo encontrado. Nenhuma linha foi removida.

No negative values found. No rows were removed.


## 4. Mapping Elements and Measurement Units

O script percorre a `element_list` fornecida, dividindo cada string em Elemento e Unidade. Em seguida, faz um cruzamento com os LODs capturados para criar uma tabela de metadados abrangente para exportação. <br>
<font color='gray'>The script iterates through the provided `element_list`, splitting each string into Element and Unit. It then cross-references this with the captured LODs to create a comprehensive metadata table for export.</font>

In [16]:
element_list = [
    'Al_pct', 'As_ppm', 'Ba_ppm', 'Be_ppm', 'Bi_ppm', 'Ca_pct', 'Cd_ppm', 'Ce_ppm', 'Co_ppm', 'Cr_ppm', 'Cs_ppm', 'Cu_ppm',
    'Fe_pct', 'Ga_ppm', 'In_ppm', 'K_pct', 'La_ppm', 'Li_ppm', 'Lu_ppm', 'Mg_pct', 'Mn_ppm', 'Mo_ppm', 'Nb_ppm', 'Pb_ppm', 'S_pct',
    'Sc_ppm', 'Se_ppm', 'Sr_ppm', 'Ta_ppm', 'Te_ppm', 'U_ppm', 'W_ppm', 'Y_ppm', 'Zn_ppm', 'Zr_ppm']

# Lista para armazenar os metadados consolidados
# List to store consolidated metadata
metadados_quimicos = []

for col in element_list:
    # Divide a string pelo underline para separar Elemento de Unidade
    # Split the string by underscore to separate Element from Unit
    partes = col.split('_')
    elemento = partes[0]
    unidade = partes[1] if len(partes) >= 2 else 'N/A'
    
    # Recupera os limites de detecção dos dicionários capturados anteriormente
    # Retrieve detection limits from the previously captured dictionaries
    # Se não houver registro para a coluna, retorna 'N/A'
    # If no record exists for the column, return 'N/A'
    lod_min = lod_min_dict.get(col, 'N/A')
    lod_max = lod_max_dict.get(col, 'N/A')
    
    # Adiciona as informações à lista
    # Add information to the list
    metadados_quimicos.append({
        'Elemento': elemento, 'Unidade': unidade,
        'LOD_Min': lod_min, 'LOD_Max': lod_max, 'Coluna_Original': col})

# Cria o DataFrame de metadados
# Create the metadata DataFrame
df_metadados = pd.DataFrame(metadados_quimicos)

# Visualização do resultado
# View result
print('--- METADATA PREVIEW | PRÉVIA DOS METADADOS ---')
print(df_metadados.head())

# Exportação para Excel com múltiplas abas
# Export to Excel with multiple sheets
with pd.ExcelWriter('Banco_Dados_Geoquimico.xlsx', engine='xlsxwriter') as writer:
    # Aba de Dados Filtrados (seu DataFrame principal limpo)
    # Sheet for Filtered Data (your clean main DataFrame)
    df.to_excel(writer, sheet_name='Dados_Filtrados', index=False)
    
    # Aba de Metadados baseada na element_list
    # Metadata sheet based on element_list
    df_metadados.to_excel(writer, sheet_name='Dicionario_Metadados', index=False)

print('\nExcel file updated with element_list metadata! | Arquivo Excel atualizado com os metadados da element_list!')

--- METADATA PREVIEW | PRÉVIA DOS METADADOS ---
  Elemento Unidade LOD_Min LOD_Max Coluna_Original
0       Al     pct    0.01     N/A          Al_pct
1       As     ppm     1.0     N/A          As_ppm
2       Ba     ppm     N/A     N/A          Ba_ppm
3       Be     ppm     N/A     N/A          Be_ppm
4       Bi     ppm    0.02     N/A          Bi_ppm

Excel file updated with element_list metadata! | Arquivo Excel atualizado com os metadados da element_list!


# 5. Save data

O processo de exportação final agora utiliza a `element_list` predefinida para construir o dicionário de metadados. Ele inclui apenas as colunas que passaram pelos filtros de qualidade anteriores, garantindo que o arquivo Excel reflita estritamente o conteúdo do banco de dados final. <br>
<font color='gray'>The final export process now uses the predefined `element_list` to build the metadata dictionary. It only includes columns that passed previous quality filters, ensuring the Excel file strictly reflects the final database's contents.</font>

In [17]:
# Lista de colunas para remover
# List of columns to remove
columns_to_remove = [
    'PROJETO_AMOSTRAGEM', 'PROJETO_PUBLICACAO', 'CENTRO_DE_CUSTO', 'CLASSE', 
    'NUMERO_DE_CAMPO', 'NUMERO_DE_LABORATORIO', 'DUPLICATA', 'LOTE', 'RA', 
    'DATA_DE_ANALISE', 'METODO', 'ABERTURA', 'LEITURA', 'LABORATORIO', 'JOB', 'OBSERVACAO']

# Remove apenas as colunas que de fato existem no DataFrame atual
# Remove only the columns that actually exist in the current DataFrame
df.drop(columns=[col for col in columns_to_remove if col in df.columns], inplace=True)

In [18]:
# Show all collumns
print(df.columns.to_list())

['LATITUDE', 'LONGITUDE', 'ID_REGISTRO', 'ID_AMOSTRA', 'Al_pct', 'As_ppm', 'Ba_ppm', 'Be_ppm', 'Bi_ppm', 'Ca_pct', 'Cd_ppm', 'Ce_ppm', 'Co_ppm', 'Cr_ppm', 'Cs_ppm', 'Cu_ppm', 'Fe_pct', 'Ga_ppm', 'In_ppm', 'K_pct', 'La_ppm', 'Li_ppm', 'Lu_ppm', 'Mg_pct', 'Mn_ppm', 'Mo_ppm', 'Nb_ppm', 'Pb_ppm', 'S_pct', 'Sc_ppm', 'Se_ppm', 'Sr_ppm', 'Ta_ppm', 'Te_ppm', 'U_ppm', 'W_ppm', 'Y_ppm', 'Zn_ppm', 'Zr_ppm', 'OBSERVACAO']


In [19]:
import pandas as pd

# 1. Lista de elementos alvo (definida pelo usuário)
# 1. Target element list (user-defined)
element_list = [
    'Al_pct', 'As_ppm', 'Ba_ppm', 'Be_ppm', 'Bi_ppm', 'Ca_pct', 'Cd_ppm', 'Ce_ppm', 'Co_ppm', 'Cr_ppm', 'Cs_ppm', 'Cu_ppm',
    'Fe_pct', 'Ga_ppm', 'In_ppm', 'K_pct', 'La_ppm', 'Li_ppm', 'Lu_ppm', 'Mg_pct', 'Mn_ppm', 'Mo_ppm', 'Nb_ppm', 'Pb_ppm', 'S_pct',
    'Sc_ppm', 'Se_ppm', 'Sr_ppm', 'Ta_ppm', 'Te_ppm', 'U_ppm', 'W_ppm', 'Y_ppm', 'Zn_ppm', 'Zr_ppm'
]

# 2. Consolidar Elementos, Unidades e LODs em uma lista
# 2. Consolidate Elements, Units, and LODs into a list
metadados_quimicos = []

# Filtramos a element_list para processar apenas o que restou no DataFrame atual
# Filter element_list to process only what remains in the current DataFrame
cols_presentes = [col for col in element_list if col in df.columns]

for col in cols_presentes:
    # Divide a string para separar Elemento e Unidade
    # Split string to separate Element and Unit
    partes = col.split('_')
    elemento = partes[0]
    unidade = partes[1] if len(partes) >= 2 else 'N/A'
    
    # Busca os valores nos dicionários capturados durante o tratamento de LOD
    # Retrieve values from the dictionaries captured during LOD treatment
    lod_min = lod_min_dict.get(col, 'N/A')
    lod_max = lod_max_dict.get(col, 'N/A')
    
    metadados_quimicos.append({
        'Elemento': elemento,
        'Unidade': unidade,
        'LOD_Min': lod_min,
        'LOD_Max': lod_max,
        'Coluna_Original': col
    })

# Criar DataFrame de metadados
# Create metadata DataFrame
df_metadados = pd.DataFrame(metadados_quimicos)

# 3. Gerar o arquivo Excel com as abas atualizadas
# 3. Generate the Excel file with updated sheets
with pd.ExcelWriter('Data/' + file_name + '_PROC.xlsx', engine='xlsxwriter') as writer:
    # Aba 1: Dados Filtrados e Convertidos (Dataset principal)
    # Sheet 1: Filtered and Converted Data (Main dataset)
    df.to_excel(writer, sheet_name='Dados_Filtrados', index=False)
    
    # Aba 2: Dicionário de Metadados (Elementos, Unidades e LODs)
    # Sheet 2: Metadata Dictionary (Elements, Units, and LODs)
    df_metadados.to_excel(writer, sheet_name='Dicionario_Metadados', index=False)
    
    # Aba 3: Registro de Duplicatas (se o objeto existir no ambiente)
    # Sheet 3: Duplicate Records (if object exists in environment)
    if 'df_duplicata' in locals():
        df_duplicata.to_excel(writer, sheet_name='Registro_Duplicatas', index=False)

print('Excel generated with complete Metadata! | Excel gerado com Metadados completos!')

Excel generated with complete Metadata! | Excel gerado com Metadados completos!
